<a href="https://colab.research.google.com/github/KumarGaurav20/Spam-Emails/blob/main/TRAINING_NAIVE_BAYES_CLASSIFIER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TRAINING NAIVE BAYES CLASSIFIER

IMPORTS

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

CONSTANTS

In [5]:
TRAINING_DATA_FILE = '/content/gdrive/MyDrive/COLAB DATA/NAIVE BAYES CLASSIFIER DATA/train-data.txt'
TEST_DATA_FILE = '/content/gdrive/MyDrive/COLAB DATA/NAIVE BAYES CLASSIFIER DATA/test-data.txt'

VOCAB_SIZE = 2500

ACESSING DATA


In [6]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)


In [14]:
sparse_train_data[0:10]

array([[ 0,  0,  1,  2],
       [ 0,  7,  1,  1],
       [ 0, 15,  1,  1],
       [ 0, 18,  1,  1],
       [ 0, 24,  1,  1],
       [ 0, 29,  1,  1],
       [ 0, 52,  1,  1],
       [ 0, 60,  1,  1],
       [ 0, 64,  1,  1],
       [ 0, 65,  1,  1]])

In [13]:
sparse_test_data[0:10]

array([[   8,    7,    1,    1],
       [   8,    8,    1,    1],
       [   8,   12,    1,    1],
       [   8,   19,    1,    2],
       [   8,   26,    1,    1],
       [   8,   56,    1,    1],
       [   8,  187,    1,    1],
       [   8,  301,    1,    1],
       [   8, 1122,    1,    1],
       [   8, 1146,    1,    1]])

FUNC FOR CONVERSION OF SPARSE MATRIX TO FULL MATRIX

In [15]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas dataframe. 
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens. 
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix